# 方策勾配 Polycy Gradient


- アクションがどれくらい効果的だったのか？

 - 試行ごとに計算、**アクションを選択する方針（方策・ポリシー）** を最適化

- どの手が有効か？の選択基準を **教師なし学習** で学ぶ

# Multi-armed Bandit Problem

- 方策勾配のサンプルとしてよく使われるのが、スロットマシン(Bandit)
- スロットマシンに複数のアームがついている
- どのアームをどの順番で

# 適用対象

限られたリソースをいかに最適配分するか？

- ギャンブル
- 研究開発投資
- 臨床試験
- コンテンツ管理

# 4-Bandits Problem

4本腕のバンディット。

- 各アームごとにあたる閾値が異なる

|0.2|-0.5|-0.2|0|

- 閾値が低いほど当たりやすい
- 0から1の乱数を発生させて都度判定

1. アームを引く
2. 報酬を得る（アタリ、はずれ）
3. 損失関数を計算する
4. 最小化するようWを更新
5. 1へ戻る

## 損失関数

\begin{equation*}
Loss = - \log \pi A
\end{equation*}

Π : アクションのウェイト
A : アドバンテージ(報酬)


In [1]:
import tensorflow as tf
import numpy as np

In [2]:
# アームの閾値 値が低いほどあたりが出やすい
bandits = [0.2, -0.5, -0.2, 0]

# アームの本数
num_bandits = len(bandits)

In [4]:
# アームの閾値を渡すとあたり(1)かはずれ(-1)かを返す
def pullBandit(bandit):
    result = np.random.randn(1)
    if result > bandit:
        return 1
    else:
        return -1

In [9]:
# TensorFlow 計算グラフ

# 計算グラフをリセット
tf.reset_default_graph

# 重み
weights = tf.Variable(tf.ones([num_bandits]))

# 何番目のアームを引くか
chosen_action = tf.argmax(weights, 0)

# 報酬を格納するための変数
reward_holder = tf.placeholder(shape=[1], dtype=tf.float32)

# アクションを保持するための変数
action_holder = tf.placeholder(shape=[1], dtype=tf.int32)

# アクションのウェイトを取り出す変数　重みから現在選択しているアクションに入っているウェイトを取り出す
responsible_weight = tf.slice(weights, action_holder, [1])

# 損失関数
loss = -(tf.log(responsible_weight)*reward_holder)

# 最適化
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)

# モデル更新
update = optimizer.minimize(loss)

In [14]:
# トレーニング

## パラメータ
# 試行回数
total_episodes = 1000
# 累積報酬
total_reward = np.zeros(num_bandits)
# 閾値 ε
e = 0.1

# TensorFlow 変数の初期化
init = tf.global_variables_initializer()

with tf.Session() as sess:
    # 変数の初期化
    sess.run(init)
    # カウンタ
    i = 0;
    while i < total_episodes:
        # 閾値より乱数が小さい場合
        if np.random.rand(1) < e:
            # ランダムにアクションを決定する
            action = np.random.randint(num_bandits)
        else:
            action = sess.run(chosen_action)
        # 報酬を計算
        reward = pullBandit(bandits[action])
        
        # モデルを更新、ウェイトの取り出し
        _,resp,ww = sess.run([update, responsible_weight, weights], feed_dict={
            reward_holder: [reward], 
            action_holder: [action]})
        
        # 累積報酬
        total_reward[action] += reward
        # 
        if i % 50 == 0:
            print("Reward list: " + str(total_reward))
        i += 1

print("Best arc on agent is " + str(np.argmax(ww) + 1) + "th arm!")

Reward list: [-1.  0.  0.  0.]
Reward list: [ -3.  19.  -2.  -1.]
Reward list: [ -5.  40.  -3.  -1.]
Reward list: [ -5.  54.  -3.  -1.]
Reward list: [ -5.  69.  -3.   0.]
Reward list: [ -5.  74.  -2.   0.]
Reward list: [ -4.  85.  -1.   1.]
Reward list: [  -4.  111.   -1.    1.]
Reward list: [  -4.  138.    0.    1.]
Reward list: [  -5.  162.   -1.    1.]
Reward list: [  -8.  184.   -2.    1.]
Reward list: [  -8.  213.   -3.    3.]
Reward list: [  -6.  231.   -2.    2.]
Reward list: [  -7.  244.   -2.    2.]
Reward list: [  -9.  269.   -3.    2.]
Reward list: [ -10.  299.   -3.    1.]
Reward list: [ -10.  303.   -5.    1.]
Reward list: [ -11.  323.   -4.    3.]
Reward list: [ -12.  338.   -2.    3.]
Reward list: [ -12.  354.   -2.    3.]
Best arc on agent is 2th arm!
